In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver  
import pandas as pd
import re

In [46]:
url = 'https://ehsan.sa/projects/0/48632?amount=null'
page = requests.get(url)
print(page.status_code)

200


In [47]:
driver = webdriver.Chrome()
# request the page 
driver.get(url)

        # extract the body and parse it
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

In [16]:
page_title = soup.title.text # gets you the text of the <title>(...)</title>
page_title

'المشاريع - منصة إحسان '

In [57]:
# extract car names
divs = soup.find_all('label', {'class':"h6"})
lis = []
for i in divs:
    lis.append(i.text)
lis

['300,000ر.س',
 '0ر.س',
 '20000 مستفيد',
 'المواطنون والمقيمون بالمملكة',
 'مكة المكرمة']

In [62]:
case_number = soup.find_all('span', {'class': 'font-ehsan-Bold text-neutral-7'})#.text.strip()
#date = soup.find_all('span', {'class': 'font-ehsan-Bold text-neutral-7'})[1].text.strip()
case_number

[]

In [58]:
# extract car names
divs = soup.find_all('span', {'class':"h5 me-2"})
lis = []
for i in divs:
    lis.append(i.text)
lis

[' 1', '16,226', '20,000']

In [80]:
# extract car names
divs = soup.find_all('p', {'class':"fs-14px h6 mb-0 ms-2 text-neutral-5"})[0].text.strip()
divs


'جمعية البر الخيرية بالأبواء'

In [21]:
# extract car names
divs = soup.find_all('a', {'class':"text-truncate card-title h6 px-2 py-1 w-max-content text-decoration-none"})
for i in divs:
    print(i.text)

سداد إيجارات المتعففين
وجبات الإفطار في ساحات الحرم المكي
وجبات السحور للصائمين
وجبات الإفطار لضيوف الرحمن
ترميم مساكن المتعففين
عملية في العمود الفقري لمريضة متعففة
سقيا الماء للأسر المتعففة
عملية في الفقرات العنقية لمريض متعفف
إفطار الصائمين


In [42]:

# extract car names
divs = soup.find_all('div', {'class':"d-flex flex-column col"})
for i in divs:
    print(i.text.split())

['رقم', 'الحالة', 'P49606']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49631']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49553']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49625']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49618']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49626']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49584']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49623']
['تاريخ', 'الاكتمال', '07/03/2025']
['رقم', 'الحالة', 'P49620']
['تاريخ', 'الاكتمال', '07/03/2025']


In [38]:

divs = soup.find_all('span', {'class':"font-ehsan-Bold text-neutral-7"})
for i in divs:
    print(i.text)

P49606
07/03/2025
P49631
07/03/2025
P49553
07/03/2025
P49625
07/03/2025
P49618
07/03/2025
P49626
07/03/2025
P49584
07/03/2025
P49623
07/03/2025
P49620
07/03/2025


In [11]:
#Define the DataFrame
columns = ['Case Number','Project Names', 'Amount', 'Date',   
             'Beneficiaries', 'Beneficiary type',  
            'Number of donations', 'Partner','Location','Details URL',]
df = pd.DataFrame(columns=columns)

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()


# Base URL
base_url = "https://ehsan.sa/completed?initiativeType=CompletedProject&p="

# Loop through multiple pages
for i in range(1, 166):  # Adjust the range as needed
    url = f"{base_url}{i}"
    driver.get(url)
    time.sleep(2)  # Wait for the page to load

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Extract project containers
    project_containers = soup.find_all('div', {'class': 'bg-neutral-s d-flex justify-content-between p-3 mb-3'})
    
    for container in project_containers:
        # Extract project name
        project_name = container.find_previous('div', {'class': 'align-items-center d-flex justify-content-between mb-3 w-100'}).text.strip()
        
        # Extract case number and date
        case_number = container.find('span', {'class': 'font-ehsan-Bold text-neutral-7'}).text.strip()
        date = container.find_all('span', {'class': 'font-ehsan-Bold text-neutral-7'})[1].text.strip()
        
        # Extract the "عرض التفاصيل" link
        details_link = container.find_next('a', text='عرض التفاصيل', href=True)
        if details_link:
            full_details_url = f"https://ehsan.sa{details_link['href']}"  # Construct full URL
        else:
            full_details_url = "Link not found"


        
        # Navigate to the details URL to scrape additional data
        if full_details_url != "Link not found":
            driver.get(full_details_url)
            time.sleep(2)  # Wait for the page to load
            details_soup = BeautifulSoup(driver.page_source, "html.parser")
            details = details_soup.find_all('label', {'class':"h6"})
            lis = []
            for i in details:
                lis.append(i.text)
                
            amount = lis[0]
            beneficiary_type = lis[3]
            location = lis[4]

            details1 = details_soup.find_all('span', {'class':"h5 me-2"})
            lis = []
            for i in details1:
                lis.append(i.text)
                
            donations = lis[1]
            beneficiaries = lis[2]

            # Extract Name of Partner
            partner = details_soup.find_all('p', {'class': 'fs-14px h6 mb-0 ms-2 text-neutral-5'})[0].text.strip()


        
        # Store the data in the DataFrame
        new_row_df = pd.DataFrame({
            'Case Number': [case_number],
            'Project Names': [project_name],
            'Date': [date],
            'Amount': amount,
            'Details URL': [full_details_url],
            'Beneficiaries': beneficiaries,
            'Beneficiary type': beneficiary_type,
            'Number of donations': donations,
            'Partner': partner,
            'Location': location,
            
            
        })
        df = pd.concat([df, new_row_df], ignore_index=True)

driver.quit()

In [29]:
df


,Case Number,Project Names,Amount,Date,Beneficiaries,Beneficiary type,Number of donations,Partner,Location,Details URL
0,P49631,وجبات الإفطار في ساحات الحرم المكي,"300,000ر.س",07/03/2025,"20,000",المواطنون والمقيمون بالمملكة,"16,226",جمعية البر الخيرية بالأبواء,مكة المكرمة,https://ehsan.sa/projects/0/48632?amount=null
1,P49553,وجبات السحور للصائمين,"280,000ر.س",07/03/2025,"20,000",المواطنون والمقيمون بالمملكة,"24,403",جمعية البر الخيرية بالنويعمة,وادى الدواسر,https://ehsan.sa/projects/0/48554?amount=null
2,P49625,وجبات الإفطار لضيوف الرحمن,"300,000ر.س",07/03/2025,"30,000",المواطنون والمقيمون بالمملكة,"11,930",جمعية البر الخيرية بالحوية,السيل الكبير,https://ehsan.sa/projects/0/48626?amount=null
3,P49618,ترميم مساكن المتعففين,"300,000ر.س",07/03/2025,11,المواطنون,"9,378",الجمعية الخيرية ببللحمر,باللحمر,https://ehsan.sa/projects/0/48619?amount=null
4,P49626,عملية في العمود الفقري لمريضة متعففة,"24,000ر.س",07/03/2025,1,المقيمون بالمملكة,145,جمعية رحماء الصحية,الرياض,https://ehsan.sa/projects/0/48627?amount=null
...,...,...,...,...,...,...,...,...,...,...
1480,P47450,توفير البطانيات للأسر المحتاجة,"103,680ر.س",25/01/2024,432,المواطنون,"7,685",جمعية الجرين الأهلية,الجرين,https://ehsan.sa/projects/0/46451?amount=null
1481,P47439,غرس الأشجار لمكافحة التصحر,"80,000ر.س",25/01/2024,"1,000",المقيمون بالمملكة,"8,673",أجيالنا التطوعية,النماص,https://ehsan.sa/projects/0/46440?amount=null
1482,P47441,تأثيث منازل المحتاجين,"75,000ر.س",25/01/2024,3,المواطنون,"4,777",جمعية البر الاهلية بباشوت,باشوت,https://ehsan.sa/projects/0/46442?amount=null
1483,P47317,دعم الأسر المنتجة,"200,000ر.س",25/01/2024,10,المواطنون,"18,792",جمعية التنمية الأهلية بالشقيق,الشقيق,https://ehsan.sa/projects/0/46318?amount=null


In [28]:
# Data from 23/4/2024
df.to_csv('data.csv')